In [25]:
!pip install -qU sentence-transformers
!pip install -qU git+https://github.com/run-llama/llama_index
!pip install -qU transformers accelerate bitsandbytes
!pip install -qU chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

# Initial pipeline - using text files to do QnA

In [26]:
from llama_index.embeddings import HuggingFaceEmbedding, AdapterEmbeddingModel
from llama_index import VectorStoreIndex, SimpleDirectoryReader, SummaryIndex, set_global_tokenizer
from IPython.display import Markdown, display
from llama_index import StorageContext, load_index_from_storage
from llama_index import ServiceContext, set_global_service_context
import chromadb
from llama_index.vector_stores import ChromaVectorStore
from llama_index.llms import HuggingFaceLLM, OpenAI
from llama_index.prompts import PromptTemplate
from transformers import AutoTokenizer
import torch
from transformers import BitsAndBytesConfig
from llama_index.response.notebook_utils import display_response
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
from llama_index.embeddings import resolve_embed_model
import torch
from llama_index.embeddings.adapter_utils import TwoLayerNN

In [27]:
# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# documents = SimpleDirectoryReader("/kaggle/input/aws-case-studies-and-blogs").load_data()

In [31]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [32]:
llm = HuggingFaceLLM(
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    query_wrapper_prompt=PromptTemplate("<s>[INST] {query_str} [/INST] </s>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.2, "top_k": 5, "top_p": 0.95},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

## Basic querying

In [7]:
query_engine = vector_index.as_query_engine(response_mode="compact")
response = query_engine.query("How are manufacturing companies in automotive industry and others using Gen AI on cloud?")
display_response(response)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:407: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
2024-02-05 06:21:54.397245: E ext

**`Final Response:`** Generative AI is being used by manufacturing companies in the automotive industry and others to transform their businesses and disrupt their industries. The potential of generative AI is incredibly exciting, but companies are still in the early days of using it. The study by IDC titled "The State of Manufacturing and Generative AI Adoption in Manufacturing Organizations" revealed that for manufacturers, the top business areas where survey respondents felt generative AI could make the most impact in the next 18 months were in manufacturing (production), product development and design, followed by sales and supply chain.

In the automotive industry, generative AI is being used to create radical, new product designs, drive unprecedented levels of manufacturing productivity, and optimize supply chain applications. For example, Autodesk, a leader in 3D design, engineering, and entertainment software, offers generative design capabilities in their Fusion 360 software to help product designers create innovative new designs within parameters specified by the user, including materials, manufacturing constraints, safety factors, and other variables.

Generative AI is also being used in the automotive industry to improve safety, create simulation datasets, explore how a part might be manufactured or machined faster, and bring

---

# Finetuning EMBEDDING:
[LlamaIndex source](https://docs.llamaindex.ai/en/stable/examples/finetuning/embeddings/finetune_embedding_adapter.html#finetuning-an-adapter-on-top-of-any-black-box-embedding-model)

### 1. Data prep

In [8]:
import json
from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SentenceSplitter
from llama_index.schema import MetadataMode

In [12]:
TRAIN_FILES = "/kaggle/input/aws-case-study-data-with-splits/data/train/input/"
VAL_FILES = "/kaggle/input/aws-case-study-data-with-splits/data/val/input/"

TRAIN_CORPUS_FPATH = "./data_samples/train_corpus.json"
VAL_CORPUS_FPATH = "./data_samples/val_corpus.json"

In [13]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    # reader = SimpleDirectoryReader(input_files=files)
    reader = SimpleDirectoryReader(input_dir=files)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SentenceSplitter()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

In [14]:
train_nodes = load_corpus(TRAIN_FILES, verbose=True)
val_nodes = load_corpus(VAL_FILES, verbose=True)

Loading files /kaggle/input/aws-case-study-data-with-splits/data_samples/train/input/
Loaded 12 docs


Parsing nodes:   0%|          | 0/12 [00:00<?, ?it/s]

Parsed 32 nodes
Loading files /kaggle/input/aws-case-study-data-with-splits/data_samples/val/input/
Loaded 8 docs


Parsing nodes:   0%|          | 0/8 [00:00<?, ?it/s]

Parsed 22 nodes


In [16]:
# !rm val_dataset.json train_dataset.json

In [17]:
from llama_index.finetuning import generate_qa_embedding_pairs, EmbeddingQAFinetuneDataset

train_dataset = generate_qa_embedding_pairs(train_nodes, llm=llm, num_questions_per_chunk=2)
val_dataset = generate_qa_embedding_pairs(val_nodes, llm=llm, num_questions_per_chunk=2)

train_dataset.save_json("train_dataset.json")
val_dataset.save_json("val_dataset.json")

100%|██████████| 22/22 [01:52<00:00,  5.11s/it]


In [18]:
# ??generate_qa_embedding_pairs

In [19]:
# len(train_dataset.queries.keys())

### 2. Finetune

In [20]:
# !ls -lrt /kaggle/input/questions-w-docs

In [57]:
train_dataset = EmbeddingQAFinetuneDataset.from_json("/kaggle/input/questions-w-docs/train_dataset.json")
val_dataset = EmbeddingQAFinetuneDataset.from_json("/kaggle/input/questions-w-docs/val_dataset.json")

In [63]:
base_embed_model = resolve_embed_model("local:BAAI/bge-large-en-v1.5")

adapter_model = TwoLayerNN(
    1024,  # input dimension
    2048,  # hidden dimension
    1024,  # output dimension
    bias=True,
    add_residual=True,
)

finetune_engine_1 = EmbeddingAdapterFinetuneEngine(
    train_dataset,
    base_embed_model,
    model_output_path="model_output_test",
    epochs=4,
    verbose=True,
    batch_size = 16
)

finetune_engine_2 = EmbeddingAdapterFinetuneEngine(
    train_dataset,
    base_embed_model,
    model_output_path="model25_output_test",
#     model_checkpoint_path="model10_ck",
    adapter_model=adapter_model,
    epochs=10,
    # bias=True,
    verbose=True,
    batch_size=32
#     optimizer_class=torch.optim.AdamW,
#     optimizer_params={"lr": 0.1}
)

# finetune_engine_3 = EmbeddingAdapterFinetuneEngine(
#     train_dataset,
#     base_embed_model,
#     model_output_path="model50_output_test",
#     model_checkpoint_path="model50_ck",
#     adapter_model=adapter_model,
#     epochs=50,
#     # bias=True,
#     verbose=True,
#     batch_size=32
# #     optimizer_class=torch.optim.AdamW,
# #     optimizer_params={"lr": 0.1}
# )

In [64]:
# finetune_engine_1.finetune()
finetune_engine_2.finetune()
# finetune_engine_3.finetune()

> Prepared optimizer, scheduler, and loss model.


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

> [Epoch 0] Current loss: 1.5551069974899292
> [Epoch 0] Current loss: 1.6143423318862915
> [Epoch 0] Current loss: 1.600243330001831
> [Epoch 0] Current loss: 1.376318097114563
> [Epoch 0] Current loss: 2.7124955654144287
> [Epoch 0] Current loss: 3.7398133277893066
> [Epoch 0] Current loss: 2.7081539630889893
> [Epoch 0] Current loss: 1.67755126953125
> [Epoch 0] Current loss: 2.5075714588165283
> [Epoch 0] Current loss: 1.520779013633728
> [Epoch 0] Current loss: 1.4745124578475952
> [Epoch 0] Current loss: 1.4212514162063599
> [Epoch 0] Current loss: 1.4936225414276123
> [Epoch 0] Current loss: 1.6054432392120361
> [Epoch 0] Current loss: 1.3321013450622559
> [Epoch 0] Current loss: 1.696689248085022
> [Epoch 0] Current loss: 1.4604642391204834
> [Epoch 0] Current loss: 1.6837546825408936
> [Epoch 0] Current loss: 1.62134850025177
> [Epoch 0] Current loss: 1.607418179512024
> [Epoch 0] Current loss: 1.6276984214782715
> [Epoch 0] Current loss: 1.5670392513275146
> [Epoch 0] Current

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

...

> [Epoch 7] Current loss: 1.5380022525787354
> [Epoch 7] Current loss: 1.5986971855163574
> [Epoch 7] Current loss: 1.5928592681884766
> [Epoch 7] Current loss: 1.360467553138733
> [Epoch 7] Current loss: 2.7095625400543213
> [Epoch 7] Current loss: 3.7475051879882812
> [Epoch 7] Current loss: 2.6925735473632812
> [Epoch 7] Current loss: 1.6606953144073486
> [Epoch 7] Current loss: 2.495100498199463
> [Epoch 7] Current loss: 1.5085843801498413
> [Epoch 7] Current loss: 1.4619048833847046
> [Epoch 7] Current loss: 1.4091418981552124
> [Epoch 7] Current loss: 1.4851795434951782
> [Epoch 7] Current loss: 1.5876985788345337
> [Epoch 7] Current loss: 1.32145094871521
> [Epoch 7] Current loss: 1.6850159168243408
> [Epoch 7] Current loss: 1.4458694458007812
> [Epoch 7] Current loss: 1.6697205305099487
> [Epoch 7] Current loss: 1.6081798076629639
> [Epoch 7] Current loss: 1.5973702669143677
> [Epoch 7] Current loss: 1.6115970611572266
> [Epoch 7] Current loss: 1.5449635982513428
> [Epoch 7] Cu

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

> [Epoch 8] Current loss: 1.5355300903320312
> [Epoch 8] Current loss: 1.5965023040771484
> [Epoch 8] Current loss: 1.5918865203857422
> [Epoch 8] Current loss: 1.3582987785339355
> [Epoch 8] Current loss: 2.709029197692871
> [Epoch 8] Current loss: 3.74870228767395
> [Epoch 8] Current loss: 2.6903669834136963
> [Epoch 8] Current loss: 1.6586024761199951
> [Epoch 8] Current loss: 2.493504524230957
> [Epoch 8] Current loss: 1.5069812536239624
> [Epoch 8] Current loss: 1.460289478302002
> [Epoch 8] Current loss: 1.4076422452926636
> [Epoch 8] Current loss: 1.4840947389602661
> [Epoch 8] Current loss: 1.585601806640625
> [Epoch 8] Current loss: 1.3201580047607422
> [Epoch 8] Current loss: 1.6835993528366089
> [Epoch 8] Current loss: 1.4440973997116089
> [Epoch 8] Current loss: 1.6680471897125244
> [Epoch 8] Current loss: 1.6066433191299438
> [Epoch 8] Current loss: 1.596306324005127
> [Epoch 8] Current loss: 1.6097240447998047
> [Epoch 8] Current loss: 1.5424525737762451
> [Epoch 8] Curre

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

> [Epoch 9] Current loss: 1.5340189933776855
> [Epoch 9] Current loss: 1.5951746702194214
> [Epoch 9] Current loss: 1.59130859375
> [Epoch 9] Current loss: 1.3570085763931274
> [Epoch 9] Current loss: 2.7086639404296875
> [Epoch 9] Current loss: 3.7494163513183594
> [Epoch 9] Current loss: 2.6890335083007812
> [Epoch 9] Current loss: 1.657394289970398
> [Epoch 9] Current loss: 2.492574691772461
> [Epoch 9] Current loss: 1.5060539245605469
> [Epoch 9] Current loss: 1.4593616724014282
> [Epoch 9] Current loss: 1.4067885875701904
> [Epoch 9] Current loss: 1.4834771156311035
> [Epoch 9] Current loss: 1.5844342708587646
> [Epoch 9] Current loss: 1.3194345235824585
> [Epoch 9] Current loss: 1.682813048362732
> [Epoch 9] Current loss: 1.4431240558624268
> [Epoch 9] Current loss: 1.6671358346939087
> [Epoch 9] Current loss: 1.6058175563812256
> [Epoch 9] Current loss: 1.5957435369491577
> [Epoch 9] Current loss: 1.6087251901626587
> [Epoch 9] Current loss: 1.5411324501037598
> [Epoch 9] Curren

In [73]:
# base_embed_model = None
# finetune_engine_1 = None
# finetune_engine_2 = None
# gc.collect()

In [79]:
!zip -r emb_ft_large_saved.zip voyage_model_output_test voyage_model25_output_test
# !du -sh ./voyage_model25_output_test/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


updating: voyage_model_output_test/ (stored 0%)
updating: voyage_model25_output_test/ (stored 0%)
  adding: voyage_model_output_test/pytorch_model.bin (deflated 7%)
  adding: voyage_model_output_test/config.json (deflated 29%)
  adding: voyage_model25_output_test/pytorch_model.bin (deflated 8%)
  adding: voyage_model25_output_test/config.json (deflated 32%)


# Using ft embeddings for responding

In [65]:
val_documents = SimpleDirectoryReader("/kaggle/input/aws-case-study-data-with-splits/data/val/input").load_data()

In [66]:
# # sampling questions
# def _sample_question():
#     import json
#     import random
#     with open("/kaggle/input/questions-w-docs/val_dataset.json", "r") as f:
#         load = json.load(f)
#     queries = load["queries"]
#     hash_key, question = random.choice(list(queries.items()))
#     key_to_relevant_doc = load["relevant_docs"][hash_key]
#     relevant_doc = load["corpus"][key_to_relevant_doc[0]]
#     print(question)
#     print(relevant_doc)
# _sample_question()

In [67]:
import json
import random
import time

In [74]:
# function to randomly sample a question from val_set, and respond to them using all embedding versions

def show_generation():
        # sampling a random question & relevant doc from val set
        with open("/kaggle/input/questions-w-docs/val_dataset.json", "r") as f:
            load = json.load(f)
        queries = load["queries"]
        hash_key, question = random.choice(list(queries.items()))
        key_to_relevant_doc = load["relevant_docs"][hash_key]
        relevant_doc = load["corpus"][key_to_relevant_doc[0]]
        print("Question sampled is:: ", question)
        print()
        
        # prepare embedding models
        base_embed_model = resolve_embed_model("local:BAAI/bge-large-en-v1.5")
        embed_model_1 = LinearAdapterEmbeddingModel(base_embed_model,
                                                    "/kaggle/input/ft-embeddings/model_output_test")
        embed_model_2_2layer = AdapterEmbeddingModel(base_embed_model,
                                                     "/kaggle/input/ft-embeddings/model10_output_test",
                                                     TwoLayerNN)
        embed_model_3_2layer = AdapterEmbeddingModel(base_embed_model,
                                                     "/kaggle/input/ft-embeddings/model50_output_test",
                                                     TwoLayerNN)
        emb_models = [base_embed_model, embed_model_1, embed_model_2_2layer, embed_model_3_2layer]

        # using every embedding model generate response
        for emb in emb_models:
            service_context = ServiceContext.from_defaults(llm=llm, embed_model=emb)
            vector_index = VectorStoreIndex.from_documents(val_documents, service_context=service_context)
    #         summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

            query_engine = vector_index.as_query_engine(response_mode="compact")
            start = time.time()
            response = query_engine.query(question)
            print(f"Time taken to generate response:: {time.time()-start}")
            print(f"Generated response using embedding: {emb} ::")
            display_response(response)

In [75]:
show_generation()

Question sampled is::  What was the main challenge faced by Zalando with its previous image management solution, and how did Amazon CloudFront help address this challenge?



OutOfMemoryError: CUDA out of memory. Tried to allocate 120.00 MiB. GPU 1 has a total capacty of 14.75 GiB of which 3.06 MiB is free. Process 2543 has 14.74 GiB memory in use. Of the allocated memory 14.44 GiB is allocated by PyTorch, and 175.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [33]:
from llama_index.embeddings import LinearAdapterEmbeddingModel
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en")
embed_model_1 = LinearAdapterEmbeddingModel(base_embed_model, "/kaggle/input/ft-embeddings/model_output_test")

embed_model_2_2layer = AdapterEmbeddingModel(
    base_embed_model,
    "/kaggle/input/ft-embeddings/model10_output_test",
    TwoLayerNN,
)

embed_model_3_2layer = AdapterEmbeddingModel(
    base_embed_model,
    "/kaggle/input/ft-embeddings/model50_output_test",
    TwoLayerNN,
)

In [ ]:
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
summary_index = SummaryIndex.from_documents(documents, service_context=service_context)